# Combine nmaps

#### In this notebook, the goal is to import the neuromaps data for glu-related maps and generate some binarized masks.

1. Import packages and maps
2. Combine receptor maps of interest

## Load Packages

In [1]:
import pandas as pd
import nilearn
from nilearn.datasets import fetch_atlas_schaefer_2018
from neuromaps.datasets import fetch_atlas
from nilearn.image import binarize_img
from nilearn import image as nimage
from neuromaps.images import (load_nifti, load_data)
from neuromaps.parcellate import Parcellater
import warnings
from neuromaps.datasets import fetch_fslr, fetch_annotation, available_annotations
from neuromaps import datasets, images, nulls, resampling, stats, transforms, parcellate
from neuromaps.parcellate import Parcellater 
from neuromaps.resampling import resample_images
from neuromaps.images import dlabel_to_gifti
import neuromaps
import nibabel as nib
import os
import numpy as np
import mayavi
from PIL import Image
import hcp_utils as hcp
import nilearn.plotting as plotting
import warnings
import nibabel as nib
import pylab as plt
from nilearn.image import resample_img
from scipy.stats import zscore
#import abagen ## This is important for certification purposes 

warnings.filterwarnings('ignore')

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


## Import Data and Set Variables

In [2]:
# List annotations of interest. 
ANNOTATIONS = [
    ('dukart2018', 'flumazenil', 'MNI152', '3mm'),   # GABA
    ('rosaneto', 'abp688', 'MNI152', '1mm'),         # mGluR1
    ('dubois2015', 'abp688', 'MNI152', '1mm'),       # mGluR2
    ('smart2019', 'abp688', 'MNI152', '1mm'),        # mGluR3
    ('sandiego2015', 'flb457', 'MNI152', '1mm'),     # D2_1
    ('smith2017', 'flb457', 'MNI152', '1mm')         # D2_2
]

               
# Generate an empty list to add transformed maps
nmaps_transformed = []
nmaps_transformed_array = []
# Set paths
outpath = "/Users/pecsok/projects/Neuromaps/pecsok_pfns/neuromaps/data/maps/"
nmda_path = "/Users/pecsok/projects/Neuromaps/pecsok_pfns/neuromaps/data/maps/PET_nifti_images/"

# Notes:
#  1. NMDA is not in available annotations list but was copied from Hansen git repo
#       Already in MNI 152 at 1mm resolution. transfer to outt
#  2. We are excluding GABA map ('norgaard2021', 'flumazenil', 'MNI152', '1mm') 
#      Only has cortical data. Consider re-adding for later analyses or future studies. 

In [28]:
# Set path for downloading neuromaps data
data_dir = '/Users/pecsok/projects/Neuromaps'

### Load in neuromaps of interest

In [8]:
# 6th gen MNI template:
mni152 = fetch_atlas('MNI152', '1mm')
mninifti = nib.load(mni152['6Asym_T1w'])
us_mninifti = resample_img(mninifti, target_affine=np.eye(3)*0.82, interpolation='nearest')

# 6th gen MNI mask
mnimask = nib.load(mni152['6Asym_brainmask'])
us_mnimask = resample_img(mnimask, target_affine=np.eye(3)*0.82, interpolation='nearest')
maskarray = us_mnimask.get_fdata()

# Fetch annotations
imgs=[]
for (src, desc, space, den) in ANNOTATIONS:
    img = datasets.fetch_annotation(source=src, desc=desc,
                                    space=space, den=den)
    imgs.append(img)


NameError: name 'length' is not defined

In [12]:
# Resample GABA
resampled_GABA=neuromaps.resampling.resample_images(src=imgs[0],src_space='MNI152',trg_space='MNI152',
                                                    trg=us_mninifti,method='linear',resampling='transform_to_trg')
# Skull Strip
print(resampled_GABA)
array = resampled_GABA[0].get_fdata()
masked_array = array * maskarray
nib.save(nib.Nifti1Image(masked_array,resampled_GABA[0].affine),outpath +'resampled_GABA_masked.nii.gz')

# Z Score
masked_values = masked_array[masked_array > 0]
zscored_values = zscore(masked_values)
zmasked_array = np.zeros_like(masked_array)
zmasked_array[masked_array > 0] = zscored_values

nib.save(nib.Nifti1Image(zmasked_array,resampled_GABA[0].affine),outpath +'resampled_GABA_zmasked.nii.gz')

(<nibabel.nifti1.Nifti1Image object at 0x32d6ae610>, <nibabel.nifti1.Nifti1Image object at 0x344713490>)


In [31]:
#### Resample mGluR maps
resampled_mglu1=neuromaps.resampling.resample_images(src=imgs[1],src_space='MNI152',trg_space='MNI152',
                                                        trg=us_mninifti,method='linear',resampling='transform_to_trg')
resampled_mglu2=neuromaps.resampling.resample_images(src=imgs[2],src_space='MNI152',trg_space='MNI152',
                                                        trg=us_mninifti,method='linear',resampling='transform_to_trg')
resampled_mglu3=neuromaps.resampling.resample_images(src=imgs[3],src_space='MNI152',trg_space='MNI152',
                                                        trg=us_mninifti,method='linear',resampling='transform_to_trg')
# Save individual maps
nib.save(nib.Nifti1Image(resampled_mglu1[0].get_fdata(),resampled_mglu1[0].affine),outpath +'resampled_mGluR1.nii.gz')
nib.save(nib.Nifti1Image(resampled_mglu2[0].get_fdata(),resampled_mglu2[0].affine),outpath +'resampled_mGluR2.nii.gz')
nib.save(nib.Nifti1Image(resampled_mglu3[0].get_fdata(),resampled_mglu3[0].affine),outpath +'resampled_mGluR3.nii.gz')

# Combine maps 
combined_mglu_array = (22*resampled_mglu1[0].get_fdata()+28*resampled_mglu2[0].get_fdata()+73*resampled_mglu3[0].get_fdata())/(22+28+73)
#nib.save(nib.Nifti1Image(combined_mglu_array,resampled_mglu1[0].affine),outpath +'resampled_combined_mGluR5.nii.gz')

# Skull Strip
masked_array = combined_mglu_array * maskarray
nib.save(nib.Nifti1Image(masked_array,resampled_GABA[0].affine),outpath +'resampled_mGluR5_masked.nii.gz')

# Z Score
masked_values = masked_array[masked_array > 0]
zscored_values = zscore(masked_values)
zmasked_array = np.zeros_like(masked_array)
zmasked_array[masked_array > 0] = zscored_values

nib.save(nib.Nifti1Image(zmasked_array,resampled_nmda[0].affine),outpath +'resampled_mGluR5_zmasked.nii.gz')

In [14]:
# Resample NMDA 
nmda = nib.load(nmda_path+'NMDA_ge179_hc29_galovic.nii.gz')
resampled_nmda=neuromaps.resampling.resample_images(src=nmda,src_space='MNI152',trg_space='MNI152',
                                                        trg=us_mninifti,method='linear',resampling='transform_to_trg')
# Skull Strip
array = resampled_nmda[0].get_fdata()
masked_array = array * maskarray
nib.save(nib.Nifti1Image(masked_array,resampled_nmda[0].affine),outpath +'resampled_NMDA_masked.nii.gz')

# Z Score
masked_values = masked_array[masked_array > 0]
zscored_values = zscore(masked_values)
zmasked_array = np.zeros_like(masked_array)
zmasked_array[masked_array > 0] = zscored_values

nib.save(nib.Nifti1Image(zmasked_array,resampled_nmda[0].affine),outpath +'resampled_NMDA_zmasked.nii.gz')

In [15]:
#### Resample D2 maps
resampled_D2_1=neuromaps.resampling.resample_images(src=imgs[4],src_space='MNI152',trg_space='MNI152',
                                                        trg=us_mninifti,method='linear',resampling='transform_to_trg')
resampled_D2_2=neuromaps.resampling.resample_images(src=imgs[5],src_space='MNI152',trg_space='MNI152',
                                                        trg=us_mninifti,method='linear',resampling='transform_to_trg')

# Save individual maps
nib.save(nib.Nifti1Image(resampled_D2_1[0].get_fdata(),resampled_D2_1[0].affine),outpath +'resampled_D2_1.nii.gz')
nib.save(nib.Nifti1Image(resampled_D2_2[0].get_fdata(),resampled_D2_2[0].affine),outpath +'resampled_D2_2.nii.gz')

# Combine maps 
combined_D2_array = (55*resampled_D2_1[0].get_fdata()+37*resampled_D2_2[0].get_fdata())/(55+37)
#nib.save(nib.Nifti1Image(combined_mglu_array,resampled_mglu1[0].affine),outpath +'resampled_combined_mGluR5.nii.gz')

# Skull Strip
masked_array = combined_D2_array * maskarray
nib.save(nib.Nifti1Image(masked_array,resampled_nmda[0].affine),outpath +'resampled_D2_masked.nii.gz')

# Z Score
masked_values = masked_array[masked_array > 0]
zscored_values = zscore(masked_values)
zmasked_array = np.zeros_like(masked_array)
zmasked_array[masked_array > 0] = zscored_values

nib.save(nib.Nifti1Image(zmasked_array,resampled_nmda[0].affine),outpath +'resampled_D2_zmasked.nii.gz')